# TimeRNNレイヤを実装する
TimeRNNレイヤは、RNNレイヤを時間方向に結合していくレイヤである

In [1]:
import numpy as np
from common.time_layers import RNN
from common.functions import sigmoid

### [演習]
* 以下のTimeRNNレイヤのクラスを完成させましょう

In [2]:
# ヒント

def test(a, b, c):
    print("a=%s"%a, "b=%s"%b, "c=%s"%c)
    return

params = [1,2,3]
test(*params) # *を変数前につけると、各引数に展開される

a=1 b=2 c=3


In [3]:
class TimeRNN:
    def __init__(self, Wx, Wh, b, stateful=False):
        """
        Wx : 入力xにかかる重み
        Wh : １時刻前のhにかかる重み
        b : バイアス
        stateful : 中間層の出力を次のミニバッチ に渡す場合はTrueにする
        """
        # パラメータのリスト
        self.params = [Wx, Wh, b]
        
        # 勾配のリスト
        self.grads = [np.zeros_like(Wx), np.zeros_like(Wh), np.zeros_like(b)]
        
        self.layers = None
        self.h, self.dh = None, None
        self.stateful = stateful

    def forward(self, xs):
        """
        順伝播計算
        xs : 配列形状は、(バッチサイズ、時間数、前層のノード数)
        """
        Wx, Wh, b = self.params
        N, T, D = xs.shape # バッチサイズ、時間数、前層のノード数
        D, H = Wx.shape # 入力層のノード数、中間層のノード数

        self.layers = []
        
        # hsは、中間層の出力hを時間方向につなげたもの
        hs = np.empty((N, T, H), dtype='f')

        # 中間層の出力hを初期化する
        if not self.stateful or self.h is None:
            self.h = np.zeros((N, H), dtype='f')

        # 時間方向に計算を進める
        for t in range(T):
            
            # RNNレイヤを定義する
            layer = RNN(*self.params) # *を変数前につけると、各引数に展開される
            
            # 時刻tのデータをRNNレイヤに入力する
            self.h = layer.forward(xs[:, t, :], self.h)
            
            # 中間層の出力hをhsに代入する
            hs[:, t, :] = self.h
            
            # レイヤを追加する
            self.layers.append(layer)

        return hs

    def backward(self, dhs):
        """
        逆伝播計算
        dhs : 各時刻における出力層からの勾配を格納した変数. 配列形状は(バッチ数、時間数、中間層のノード数)
        """
        
        Wx, Wh, b = self.params
        N, T, H = dhs.shape # バッチサイズ、時間数、中間層のノード数
        D, H = Wx.shape # 前層のノード数、　中間層のノード数

        # dxsを初期化する. dxsは、各時刻におけるdxを格納する変数
        dxs = np.empty((N, T, D), dtype='f') # バッチ数、時間数、前層のノード数
        
        # dhの初期値
        dh = 0
        
        # 勾配の初期値
        grads = [0, 0, 0] #Wxの勾配、 Whの勾配、 bの勾配
        
        # 時間方向と逆向きに計算を進める
        for t in reversed(range(T)):
            
            # RNNレイヤの呼び出し
            layer = self.layers[t]
            
            # RNNレイヤの逆伝播計算
            # RNNレイヤに入力される勾配は、2方向から来るので、2つの値を足す
            dx, dh = layer.backward(dhs[:, t, :] + dh) 

            # dxをdxsに格納する
            dxs[:, t, :] = dx

            # Wxの勾配、 Whの勾配、 bの勾配、をそれぞれ足し合わせる
            for i, grad in enumerate(layer.grads):
                grads[i] += grad

        print("grads=",grads)
        # Wxの勾配、 Whの勾配、 bの勾配、を保持しておく
        for i, grad in enumerate(grads):
            self.grads[i] = grad
            
        # 最後の中間層のdhを保持しておく
        self.dh = dh

        return dxs

In [5]:
D = 1 # 前層のノード数
H = 5 # 中間層のノード数
Wx = np.random.randn(D, H)
Wh = np.random.randn(H, H)
b = np.zeros(H)

# オブジェクトの生成
time_rnn = TimeRNN(Wx, Wh, b)

# 順伝播計算
N = 4 # バッチサイズ
T = 5 # 時間数
xs = np.random.randn(N, T, D)
hs = time_rnn.forward(xs)
print("hs=", hs)
print()

# 逆伝播計算
dhs = np.random.randn(N, T, H)
dxs = time_rnn.backward(dhs)
print("dxs=", dxs)
print()


hs= [[[ 0.6347608  -0.14874917 -0.95756525  0.40289426  0.20163491]
  [-0.99764514  0.94244653  0.97385055 -0.8800474   0.7889045 ]
  [ 0.7305003  -0.85364175 -0.817177    0.99021584  0.79098237]
  [-0.99925727  0.9133757   0.9918995  -0.9979934   0.99966884]
  [-0.31967902 -0.783782    0.95199347  0.9829015   0.8733945 ]]

 [[-0.8376165   0.2379614   0.99596083 -0.598904   -0.31938705]
  [ 0.9974077  -0.9284921  -0.4949646   0.89497375 -0.95326275]
  [ 0.98077375 -0.14883669 -0.9999287  -0.82518744 -0.8154354 ]
  [ 0.7623028   0.7680383  -0.8855041   0.9986121  -0.99908286]
  [-0.9998043   0.93875563  0.9969328  -0.9960258  -0.9977451 ]]

 [[ 0.17841822 -0.03605192 -0.43095458  0.10243085  0.04916265]
  [ 0.36277488  0.45969567 -0.99696296  0.42960927  0.62108564]
  [-0.9995315   0.96410674 -0.01816249 -0.77288526  0.98310524]
  [-0.9144863   0.67380667  0.68091226  0.92141914  0.96547276]
  [-0.9869876  -0.3471973   0.39440405 -0.9982901   0.999511  ]]

 [[ 0.83271587 -0.2349024  -0.